In [ ]:
#@title Importing necessary libraries
import numpy as np
import pandas as pd
import math
import random

In [ ]:
#@title Generate Test Case
def generate_with_density(n_users: int, n_movies: int, density=0.5):
  '''
  This function generates matrices of given shape and density. The density is
  not exact, but close to the desired level. Output matrix is roughy as dense
  as we want it to be.

  A matrix with density of 30% means there are 30% actual numerical entries,
  and 70% are NaN values.

  Input:
  ------
    n_users, n_movies: int
      the shape of the matrix, n rows x m columns

    density (optional, default is 0.5): float
      given value must be between 0 and 1 (inclusive). a density of 0.3 means
      ~ 30% of the matrix is numerical value. a density of 1 means the matrix
      has nearly no NaN value.

  Output:
  ------
    A matrix (type: pandas DataFrame) of size (n_users x n_movies) with density roughly the same as
    specified value

  '''
  if (density < 0) or (density > 1):
    raise TypeError("Density must be a floating point value from 0 to 1 (inclusive)")

  # number of non-NaN values
  number_entry = int(round(density * n_users * n_movies))

  # generate a full nan matrix & convert to numpy array
  M = []
  nan_array = [np.nan] * n_movies
  for i in range(n_users):
    M.append(nan_array)
  matrix = np.asarray(M)

  # getting random entry index & replace it with numerical value
  row_idx = [*range(n_users)]
  col_idx = [*range(n_movies)]
  ratings = [*range(1,11)]
  for i in range(number_entry):
    rand_row = random.choice(row_idx)
    rand_col = random.choice(col_idx)
    matrix[rand_row, rand_col] = random.choice(ratings)

  headers = list()
  for i in range(n_movies):
    movie = "movie " + str(i)
    headers.append(movie)


  matrix = pd.DataFrame(matrix, columns=headers)
  return matrix

## Problem: Movie Suggestion

### Subprolem 1: A submatrix contains no N/A values

In [ ]:
#@title Preprocessing
def user_threshold(M, m0):
  """
  This program selects all users who have watched at least m0 movies.

  Input
  ------
    M : pandas DataFrame (2D array)
        Movie rating matrix, each row is one user, each column is one movie.
    m0 :int
        Number of minimum movie each selected user have to watch.

  Output
  ------
    A submatrix (type: pandas DataFrame) where no user has watched less than m0 movies.

  Time complexity: Θ(n x m)

  """
  n_users = M.shape[0]
  n_movies = M.shape[1]

  # list to store index of users watched at least m0 movies
  valid = list()

  # count number of movies watched by each user
  for user in range(n_users):
    watched = 0
    for movie in range(n_movies):
      if not np.isnan(M.iloc[user, movie]):
        watched += 1

    # remember users watched at least m0 movies
    if watched >= m0:
      valid.append(user)

  subM = M.iloc[valid,:]

  return subM

In [ ]:
#@title Sub-prolem 1
def subproblem1(M):
  """
  This program finds a submatrix with no NaN value and at least m0 columns. A
  random user who has watched at least m0 films is selected, then find all users
  who have watched the same films as the randomized user.

  Input
  ------
    M : pandas DataFrame (2D array)
        Preprocessed matrix using user_theshold() function.

  Output
  ------
    A submatrix (type: pandas DataFrame) with no NaN value and at least m0 columns.

  Time complexity: O(n x m)
  """
  n_users = M.shape[0]
  n_movies = M.shape[1]
  # randomize an user among those watched at least m0 movies
  standard_user = random.randrange(0, n_users)

  # find the index of movies that standard_user has watched
  standard_movies_i = list()
  for movie in range(n_movies):
    rating = M.iloc[standard_user, movie]
    if not np.isnan(rating):
      standard_movies_i.append(movie)

  # find all users who have watched the same movies as standard_user
  valid_users = list()
  for user in range(n_users):
    movies_rating = M.iloc[user, standard_movies_i]

    valid = True
    for rating in movies_rating: # worst case: standard_movies_rating = n_movies
      if np.isnan(rating):
        valid = False
        continue

    if valid:
      valid_users.append(user)

  subM = M.iloc[valid_users, standard_movies_i]

  return subM

### Sub-problem 2: The largest submatrix contains no pair-wise correlation coefficient < $\delta$
- **Input:** A dense matrix $M$: each row is movie ratings of 1 user, each column is a movie.
- **Output:** A sub-matrix of the given matrix $M$ where there is no pair-wise Correlation Coefficient < $\delta$

In [ ]:
#@title Produce Correlation Coefficient Matrix
def corr_coef(M, delta=None, boolean=False):
  """
  This function produces a pair-wise Correlation Coefficient matrix. Input
  matrix should not contain any NaN entry.

  In order for the function to return a boolean correlation matrix:
  - The `dela` parameter must be between 1 and -1 (inclusive).
  - The `boolean` parameter must be set as `True`.

  Input
  ------
    matrix: pandas DataFrame (2D array)
      A dense matrix where each row is movie ratings of an user, and
      each column is a movie.

    delta : float (Optional, default=None)
      A value between - and -1 (inclusive) must be given to be the
      correlation threshold.

    boolean:bool (Optional, default=False)
      Set to True if desired output is a correlation matrix where each
      entry has been compared with `delta`.

  Output
  ------
    If boolean=False:
      Return correlation matrix (type: numpy array) of the rows of input matrix. The Pearson correlation
      coefficient is calculated based on `corrcoef` method of Numpy. For more information,
      see https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html

    If boolean=True:
      A nxn matrix (type: numpy array) of boolean entries. If entry at position i-j in the output
      matrix is True, row i and row j in the input matrix has correlation coefficient
      >= delta. If i-j entry is False, row i and row j of input matrix has correlation
      coefficient < delta.

  Time complexity: O(n^2 x m)

  """
  matrix = M.to_numpy()
  corr_matrix = np.corrcoef(matrix) # time complexity of np.corrcoef() is (assumed to be) O(n^2 x m)
  if delta is not None:
    bool_matrix = corr_matrix >= delta

  if boolean:
    return bool_matrix
  else:
    return corr_matrix

In [ ]:
#@title Count the number of pair-wise Corr Coefs which are below `delta` of each user

def count_false(bool_matrix):
  """
  Count number of false (pairwise correlation coefficient < delta) for each user.

  Input
  ------
    bool_matrix: numpy array (2D array)
      A nxn square boolean matrix.

  Output
  ------
    A list contains n elements where each element is the number of cases
    that violate the correlation coefficient constraint (as False in the input matrix).

  Time complexity: Θ(n^2)
  """
  n = bool_matrix.shape[0]
  lst = list()
  for i in range(n): # for each user
    this_row = bool_matrix[i]
    count = 0

    # iterate over current user's boolean pair-wise correlation coefficients
    for j in range(n):
      if not this_row[j]:
        count += 1
    lst.append(count)

  return lst

In [ ]:
#@title Check if the boolean matrix has no False entry
def check_if_valid(lst):
  """
  Check if the current matrix has no correlation coefficients below the
  threshold sigma (or no "False" entry in boolean correlation matrix)

  Input
  ------
    lst:  list (1D array)
    A list storing number of False entries in each row of the boolean
    correlation matrix.

  Output
  ------
    Returns True if no user pairs violate the correlation coefficient constraint.
    Return False otherwise.

  Time complexity: Θ(n)
  """
  n = len(lst)

  for i in range(n):
    if lst[i] != 0:
      return False

  return True

In [ ]:
#@title Eliminate the user who has the largest number of pair-wise Corr Coefs below `delta`
def del_max_false(M, corr_matrix_bool, false_lst): # O(n)
  """
  This function eliminate the user who has the largest number of False
  correlation coefficient.

  Input
  ------
    M: pandas DataFrame (2D array)
       Movie rating matrix, each row is one user, each column is one movie.

    corr_matrix_bool: numpy array (2D array)
            A square boolean matrix.

    false_lst: list (1D array)
          A list storing number of False entries in each row of the boolean
          correlation matrix.

  Output
  ------
    The remaining matrix (type: pandas DataFrame) after the user with most number of False correlation
    coefficient has been eliminated.

  Time complexity: Θ(n)
  """
  max_false = 0
  max_false_id = 0
  n_users_false = len(false_lst)

  # find index of the user who has max_false
  for i in range(n_users_false):
    if max_false < false_lst[i]:
      max_false = false_lst[i]
      max_false_id = i

  # extract index of all current users except the one with most false
  n_users = corr_matrix_bool.shape[0]
  valid = [r for r in range(n_users) if r != max_false_id]

  # select all current users except the one with most False
  corr_matrix_bool = corr_matrix_bool[valid][:, valid]
  matrix = M.iloc[valid, :]

  return corr_matrix_bool, matrix

In [ ]:
#@title Sub-problem 2
def subproblem2(M, delta):
  """
  Input
  ------
    M : pandas DataFrame (2D array)
      A dense matrix (no NaN entry). Each row is an user, each column is a movie.

    delta : float
      A value between -1 and 1 (inclusive), the minumum correlation coefficient
      value.

  Output
  ------
    A submatrix (type: pandas DataFrame) in which all pairwise correlation coefficient
    are at least delta.

  Time complexity: O(n^3)
    Note: The full time complexity of this function is (n^2 x m + n^3), but for this
    problem, n is usually larger than m (more users in the database than the number of
    movies), that's why we consider n^3 to be the dominant term.

  """
  bool_corr_matrix = corr_coef(M, delta=delta, boolean=True) #O(n^2 x m)
  false_lst = count_false(bool_corr_matrix)
  status = check_if_valid(false_lst)

  # continuously deleting users with the most False in bool_corr_matrix
  # stops until bool_corr_matrix contains all True
  while not status:  # worst case n
    bool_corr_matrix, M = del_max_false(M, bool_corr_matrix, false_lst) # O(n)
    false_lst = count_false(bool_corr_matrix) # O(n^2)
    status = check_if_valid(false_lst) # O(n)

  return M

### Merged Program

In [ ]:
def movie_suggestion(M, m0, delta, n_iter):
  '''
  This is the improved program to solve the movie suggestion problem.

  Input
  ------
    M : pandas DataFrame (2D array)
      A dense matrix (no NaN entry). Each row is an user, each column is a movie.

    m0 :  int
      Number of minimum movie each selected user have to watch.

    delta : float
      A value between -1 and 1 (inclusive), the minumum correlation coefficient
      value.

    n_iter : int
      Number of iterations to run the program. The more iterations, the higher
      chance of getting a larger submatrix.

  Output
  ------
    Returns 2 lists that are subsets of rows and columns of the given matrix
    that satisfies all constraints of the problem.

    Return 2 empty lists if none such subsets found.

  Time complexity: O(n_iter x (mxn + n^3)) (Polynomial time)
  '''
  # Pre-processing
  M = user_threshold(M, m0)

  # If resulting matrix after preprocessing has less than 2 users,
  # tell user to try different input
  if M.shape[0] < 2:
    print("Cannot produce any submatrix. Try another set of input.")
    return ([], [])

  # Go! :D
  largest_subM = None
  largest_size = 0

  for iter in range(n_iter):
    # getting matrix without NaN value
    subM_no_na = subproblem1(M)

    # if resulting matrix has more than 1 user, continue with subproblem 2 to
    # check corr coef condition
    if subM_no_na.shape[0] > 1:
      subM_delta = subproblem2(subM_no_na, delta=delta)
      current_size = subM_delta.shape[0]*subM_delta.shape[1]

      if current_size > largest_size:
        largest_size = current_size
        largest_subM = subM_delta

  # check one last time that resulting submatrix has at least 2 users
  if (largest_subM is None) or (largest_subM.shape[0] < 2):
    print("Cannot produce any submatrix. Try another set of input.")
    return ([], [])
  else:
    return (largest_subM.index.tolist(), largest_subM.columns.tolist())

In [ ]:
def run(n_users, n_movies, m0, delta, data=None, density=0.5,
        n_iter=1, return_result=False):
  '''
  Wrapper function for the entire Movie Suggestion problem algorithm. Use this
  function when testing the program.

  Input
  ------
    n_users, n_movies: int
      the shape of the matrix, n rows x m columns

    m0 :  int
      Number of minimum movie each selected user have to watch.

    delta : float
      A value between -1 and 1 (inclusive), the minumum correlation coefficient
      value.

    data: (optional, default=None) pandas DataFrame
      If you want to run this program on your own data, pass the data in pandas
      DataFrame type into this function. Otherwise, a random database of specified
      density will be generated.

    density: (optional, default is 0.5): float
      given value must be between 0 and 1 (inclusive). a density of 0.3 means
      30% of the matrix is numerical value. a density of 1 means the matrix
      has no NaN value

    n_iter : int
      Number of iterations to run the program. The more iterations, the higher
      chance of getting a larger submatrix.

    return_result: (optional, default=False) bool
      Set to True if you want th final result to be returned. Otherwise nothing
      will be returned and the output will be printed to the screen.

  Output
  ------
    Subset of rows and columns will be printed to the screen. If
    return_result=True, function will return the final result.
  '''
  if data is None:
    M = generate_with_density(n_users, n_movies, density)
  else:
    M = data
  print(f"The input matrix M:\n{M}\n")

  result = movie_suggestion(M, m0, delta, n_iter)
  print(f"Subset of users: {result[0]}")
  print(f"Subset of movies: {result[1]}")

  if return_result:
    return result

## Testing

In [ ]:
#1 Test with no submatrix can be returned
M = generate_with_density(n_users=2, n_movies=2, density=0)
M = pd.DataFrame(M)

run(n_users=2, n_movies=2, m0=3, delta=0.01, data=M, density=0,
    n_iter=1, return_result=False)

The input matrix M:
   movie 0  movie 1
0      NaN      NaN
1      NaN      NaN

Cannot produce any submatrix. Try another set of input.
Subset of users: []
Subset of movies: []


In [ ]:
#2 Test with no submatrix can be returned
M = np.array([[1,2],
              [3, np.nan]])
M = pd.DataFrame(M)

run(n_users=2, n_movies=2, m0=2, delta=0.01, data=M, density=0,
    n_iter=20, return_result=False)

The input matrix M:
     0    1
0  1.0  2.0
1  3.0  NaN

Cannot produce any submatrix. Try another set of input.
Subset of users: []
Subset of movies: []


In [ ]:
#3 Test with size=100x30 and density=0.9, number of iterations=300
run(n_users=100, n_movies=30, m0=5, delta=0.01, data=None,
    density=0.9, n_iter=300, return_result=False)

The input matrix M:
    movie 0  movie 1  movie 2  movie 3  movie 4  movie 5  movie 6  movie 7  \
0       6.0      NaN      NaN      NaN      4.0      7.0      9.0      9.0   
1       1.0      1.0      1.0      3.0      NaN      1.0      NaN      1.0   
2       NaN      3.0      9.0      NaN     10.0      NaN      NaN      NaN   
3       8.0      4.0      7.0      NaN     10.0      3.0      NaN      3.0   
4      10.0      NaN      9.0     10.0      3.0      4.0      NaN      9.0   
..      ...      ...      ...      ...      ...      ...      ...      ...   
95      NaN      7.0      2.0      2.0      NaN      NaN      3.0      4.0   
96      NaN      7.0      7.0      4.0      8.0      NaN      8.0      3.0   
97      NaN      6.0      NaN      4.0      1.0      NaN      NaN      NaN   
98      NaN      3.0      3.0      3.0      4.0      4.0      NaN      3.0   
99      3.0      NaN      2.0      6.0      7.0      1.0      NaN      1.0   

    movie 8  movie 9  ...  movie 20  movie 

In [ ]:
#5 Test with size=3000x1000 and density=0.5, number of iterations=300
run(n_users=3000, n_movies=1000, m0=5, delta=0.01, data=None,
    density=0.5, n_iter=30, return_result=False)

The input matrix M:
      movie 0  movie 1  movie 2  movie 3  movie 4  movie 5  movie 6  movie 7  \
0         NaN      NaN      4.0      NaN      4.0      NaN      NaN      NaN   
1         NaN      NaN      NaN      2.0      NaN      NaN      NaN      4.0   
2         NaN      1.0      8.0      NaN      1.0      NaN      NaN      NaN   
3         6.0      NaN      8.0      NaN     10.0      NaN      NaN      4.0   
4         NaN      6.0      NaN      NaN      NaN      NaN      NaN      1.0   
...       ...      ...      ...      ...      ...      ...      ...      ...   
2995      NaN      NaN      NaN      NaN      NaN      NaN      2.0      1.0   
2996      7.0     10.0      8.0      NaN      NaN      NaN      NaN      3.0   
2997      NaN      NaN      NaN      4.0      NaN     10.0      NaN      5.0   
2998      3.0      NaN      NaN      NaN      NaN      7.0      9.0      NaN   
2999      NaN      NaN      1.0      NaN      NaN     10.0      NaN      5.0   

      movie 8  movi

In [ ]:
run(n_users=10, n_movies=5, m0=3, delta=0.01, data=None, density=1,
    n_iter=10, return_result=False)

The input matrix M:
   movie 0  movie 1  movie 2  movie 3  movie 4
0      8.0      9.0      6.0      NaN      NaN
1      NaN      NaN      NaN      NaN     10.0
2      NaN      9.0      2.0      1.0      2.0
3      6.0     10.0      5.0      5.0      NaN
4      4.0      NaN      6.0      5.0      6.0
5      NaN      5.0      1.0      2.0      NaN
6      3.0      6.0      5.0      NaN      3.0
7      NaN      NaN      NaN      1.0      3.0
8      5.0      4.0      NaN      5.0      4.0
9      8.0      NaN      NaN     10.0      NaN

Subset of users: [2, 3, 5]
Subset of movies: ['movie 1', 'movie 2', 'movie 3']
